# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
exported_file = "Output/weather_data.csv"
weather_data_df = pd.read_csv(exported_file)
weather_data_df.head()

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Country
0,Chernyshevskiy,63.0128,112.4714,31.87,72,11,2.71,RU
1,Popondetta,-8.7537,148.2534,75.36,93,100,1.03,PG
2,Impfondo,1.6381,18.0667,71.56,98,100,1.45,CG
3,Rikitea,-23.1203,-134.9692,75.92,64,76,9.98,PF
4,Port Elizabeth,-33.9180,25.5701,60.80,82,0,20.71,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [31]:
locations = weather_data_df[['Latitude','Longitude']]
weight = weather_data_df['Humidity (%)'].astype(float)
max_humidity = weight.max()

In [32]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=weight,
                                 dissipating=False,
                                 max_intensity=max_humidity,
                                 point_radius=2)
#add heat layer
fig.add_layer(heat_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
ideal_weather_df = weather_data_df.loc[(weather_data_df['Temperature']>=70) & (weather_data_df['Temperature']<=80) & (weather_data_df['Wind Speed (mph)'] < 10) & (weather_data_df['Cloudiness (%)'] == 0), :]
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df['index']
ideal_weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Country
0,Inhambane,-23.8650,35.3833,72.88,78,0,4.90,MZ
1,Charters Towers,-20.1000,146.2667,77.97,44,0,9.53,AU
2,Kyabram,-36.3167,145.0500,75.00,40,0,0.22,AU
3,Carnarvon,-24.8667,113.6333,73.40,49,0,5.75,AU
4,Guapó,-16.8306,-49.5319,71.60,73,0,4.61,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_list = []

for x in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[x]['Latitude']
    lng = ideal_weather_df.loc[x]['Longitude']
    
    params = {'location':f'{lat},{lng}',
          'radius':5000,
          'types':'hotel',
          'key': g_key}
    
    response = requests.get(url, params=params).json()
    try:
        hotel_list.append(response['results'][0]['name'])
    except:
        pass
print(hotel_list)

['Maxixe', 'Charters Towers', 'Kyabram', 'Carnarvon', 'Guapó', 'Nantong', 'Port Pirie', 'Poum', 'José de Freitas', 'Maxixe', 'Ürümqi', 'Morondava']


In [39]:
ideal_weather_df['Hotel Name'] = hotel_list
hotel_df = ideal_weather_df.dropna(how='any')
hotel_df

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Hotel Name
0,Inhambane,-23.8650,35.3833,72.88,78,0,4.90,MZ,Maxixe
1,Charters Towers,-20.1000,146.2667,77.97,44,0,9.53,AU,Charters Towers
2,Kyabram,-36.3167,145.0500,75.00,40,0,0.22,AU,Kyabram
3,Carnarvon,-24.8667,113.6333,73.40,49,0,5.75,AU,Carnarvon
4,Guapó,-16.8306,-49.5319,71.60,73,0,4.61,BR,Guapó
5,Juegang,32.3174,121.1855,71.01,55,0,1.01,CN,Nantong
6,Port Pirie,-33.1833,138.0167,71.60,40,0,6.91,AU,Port Pirie
7,Poum,-20.2333,164.0167,75.33,51,0,2.42,NC,Poum
8,José de Freitas,-4.7564,-42.5756,78.80,94,0,3.44,BR,José de Freitas
9,Maxixe,-23.8597,35.3472,71.46,81,0,4.00,MZ,Maxixe


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
location = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(location)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))